In [ ]:
import os, json, pickle
from os.path import join as ojoin

In [ ]:
sim_dir = ojoin('..', 'simulation_results') 
graph_dir = ojoin('..', '..', '..', '..', 'code', 'fairim', 'networks')


In [ ]:
def extract_min_coverage(fpath, gpath, attribute, values):
    coverages = []
    with open(fpath) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            coverages.append(float(line))
    assert len(coverages) == 500
    
    # exclude the cases where all simulated influences are zero
    if sum(coverages) == 0:
        return None
    
    with open(gpath, 'rb') as f:
        graph = pickle.load(f)
    
    relative_group_coverages = []
    for value in values:
        subgroup_indices = [nodeid for nodeid, attr_dict in graph.nodes(data=True) 
                            if attr_dict[attribute] == value]    
        rgc = sum([coverages[i] for i in subgroup_indices]) / len(subgroup_indices)
        relative_group_coverages.append(rgc)
        
    return min(relative_group_coverages)

In [ ]:
def get_min_coverage_dict():
    min_coverage_dict = dict()

    for attribute in ('age', 'gender', 'ethnicity'):
        values = attribute_values[attribute]
        mc_list = []
        for graph_number in range(20):
            fpath = ojoin(sim_dir, 'tim', 'output_%d.txt'%graph_number)
            gpath = ojoin(graph_dir, 'graph_spa_500_%d.pickle'%graph_number)
            min_coverage = extract_min_coverage(fpath, gpath, attribute, values)
            mc_list.append(min_coverage)
        min_coverage_dict[('tim', attribute)] = mc_list
        
    for problem in ('group_maximin', 'dc', 'equality', 'equity'):
        for attribute in ('age', 'gender', 'ethnicity'):
            output_dir = ojoin(sim_dir, 'mip', problem, attribute)
            values = attribute_values[attribute]
            mc_list = []
            for graph_number in range(20):
                fpath = ojoin(output_dir, 'output_%d.txt'%graph_number)
                gpath = ojoin(graph_dir, 'graph_spa_500_%d.pickle'%graph_number)
                min_coverage = extract_min_coverage(fpath, gpath, attribute, values)
                if min_coverage is not None:
                    mc_list.append(min_coverage)
            min_coverage_dict[problem, attribute] = mc_list
    
    return min_coverage_dict

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np



def draw(values, labels, colors, title, ylabel, fpath, ylim=None, figsize=None):
    if figsize is None:
        figsize=(3.5, 2)
    plt.figure(figsize=figsize)
    means = [np.mean(v) for v in values]
    stds = [np.std(v) for v in values]
    xpos = range(len(labels))

    bars = plt.bar(xpos, means, yerr=stds, width=0.5, align='center', capsize=4)
    for i in xpos:
        bars[i].set_color(colors[i])
    _ = plt.xticks(xpos, labels, rotation=0, fontsize=10)
    if ylim:
        _ = plt.ylim(ylim)
    _ = plt.ylabel(ylabel, fontsize=10)
    _ = plt.title(title, fontsize=12)
    _ = plt.savefig(fpath, bbox_inches='tight')
    

In [ ]:
with open('attribute_values.json') as f:
    attribute_values = json.load(f)
min_coverage_dict = get_min_coverage_dict()

labels = ['tim', 'equality', 'equity', 'maximin', 'diversity']
colors = ['#ff85d0'] + ['#8fffa0'] * 4

### !!! NOTE! The labels and names of directories do not match !!!
for attribute in ('age', 'gender', 'ethnicity'):
    values = [np.array(min_coverage_dict[(method, attribute)]) 
              for method in ('tim', 'equity', 'equality', 'group_maximin', 'dc')]
    title = attribute[0].upper() + attribute[1:]
    fpath = 'images/min_coverage/%s.pdf'%attribute
    draw(values, labels, colors, title, 'Minimum Relative Coverage', fpath, figsize=(4, 2.5))